#### PART 1
#### Inspect the values for each financial metric

In [ ]:
import psycopg2
import pandas as pd
from utils.utils import get_postgres_connection


def read_financial_metrics():
    """Simple function to read financial_metrics table and return as pandas DataFrame"""
    conn = get_postgres_connection()
    query = "SELECT * FROM raw.financial_metrics"
    df = pd.read_sql_query(query, conn)
    conn.close()
    return df


df = read_financial_metrics()
print(f"Read {len(df)} records from financial_metrics table")
print(df.head())


In [ ]:
import pandas as pd
import numpy as np
from IPython.display import display_html

def _safe_numeric_series(s: pd.Series) -> pd.Series:
    """Keep only finite numeric values."""
    s = pd.to_numeric(s, errors="coerce")
    s = s.replace([np.inf, -np.inf], np.nan).dropna()
    return s

def _summary_with_custom_percentiles(s: pd.Series, percentiles=None) -> pd.DataFrame:
    if percentiles is None:
        # include 5th, 95th and deciles 10..90
        percentiles = [0.001, 0.01,0.05] + [i/100 for i in range(10, 100, 10)] + [0.95, 0.99, 0.999]
    desc = s.describe(percentiles=percentiles)
    out = desc.reset_index()
    out.columns = ["Statistic", "Value"]
    return out

def _bucket_counts_qcut(s: pd.Series, q=10) -> pd.DataFrame:
    """Counts per percentile bucket, safe against duplicates and constants."""
    n = len(s)
    if n == 0:
        return pd.DataFrame({"Percentile Bucket": [], "Count": []})
    if s.nunique(dropna=True) < 2:
        return pd.DataFrame({"Percentile Bucket": ["All values equal"], "Count": [n]})

    try:
        buckets = pd.qcut(s, q=q, labels=[f"{i*100//q}-{(i+1)*100//q}%" for i in range(q)], duplicates="drop")
        counts = buckets.value_counts().sort_index().reset_index()
        counts.columns = ["Percentile Bucket", "Count"]
        return counts
    except Exception:
        qs = np.linspace(0, 1, q + 1)
        edges = np.unique(s.quantile(qs).values)
        if len(edges) < 2:
            return pd.DataFrame({"Percentile Bucket": ["All values equal"], "Count": [n]})
        labels = [f"{int(qs[i]*100)}-{int(qs[i+1]*100)}%" for i in range(len(edges)-1)]
        buckets = pd.cut(s, bins=edges, include_lowest=True, labels=labels, duplicates="drop")
        counts = buckets.value_counts().sort_index().reset_index()
        counts.columns = ["Percentile Bucket", "Count"]
        return counts

def _display_side_by_side(dfs: list, titles: list):
    html = ""
    for df, title in zip(dfs, titles):
        html += (
            "<div style='display:inline-block; padding-right:30px; vertical-align:top;'>"
            f"<h3 style='margin:4px 0 8px 0;'>{title}</h3>"
            f"{df.to_html(index=False)}"
            "</div>"
        )
    display_html(html, raw=True)

def describe_all_numeric_with_buckets(df: pd.DataFrame, q=10, percentiles=None, max_cols=None):
    """
    For each numeric column:
      - show summary with custom percentiles (5th, 10..90, 95th by default)
      - show counts per percentile bucket
    """
    num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    if max_cols is not None:
        num_cols = num_cols[:max_cols]

    if not num_cols:
        print("No numeric columns found.")
        return

    for col in num_cols:
        s = _safe_numeric_series(df[col])
        if s.empty:
            print(f"📭 Column: {col} — skipped (no finite numeric values).")
            continue

        stats = _summary_with_custom_percentiles(s, percentiles=percentiles)
        buckets = _bucket_counts_qcut(s, q=q)

        print(f"📊 Column: {col} (n={len(s)})")
        _display_side_by_side(
            [stats, buckets],
            ["Summary Statistics", f"Counts per Percentile Bucket ({q} buckets)"]
        )
        print()  # spacing


# ---- Run it ----
# This will include 5th, 10..90, 95th percentiles + decile bucket counts
describe_all_numeric_with_buckets(df, q=10)


#### Part 2 - Query against Financial Metrics

In [ ]:
import time
import psycopg2
from sqlalchemy import create_engine
import polars as pl
import pandas as pd
from datetime import datetime, date
from typing import Union


HOST = "91.107.196.130"
DATABASE = "investor_lab_db"
USERNAME = "investor_lab_user"
PASSWORD = "investor_lab_password"
PORT = 5432

def get_remote_postgres_connection():
    """Get a direct PostgreSQL connection to the remote database"""
    return psycopg2.connect(
        dbname=DATABASE,
        user=USERNAME,
        password=PASSWORD,
        host=HOST,
        port=PORT
    )

def get_sqlalchemy_engine():
    """SQLAlchemy engine (recommended for Pandas queries)"""
    db_url = f"postgresql+psycopg2://{USERNAME}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
    return create_engine(db_url)


def run_query(query: str) -> pd.DataFrame:
    """Execute a SQL query and return a Pandas DataFrame using SQLAlchemy"""
    engine = get_sqlalchemy_engine()

    start = time.time()
    df = pd.read_sql_query(query, engine)
    duration = round(time.time() - start, 2)
    print(f"✅ Query + fetch duration: {duration} seconds")

    return df


def run_query_to_polars_simple(query):
    print("start")
    conn = get_remote_postgres_connection()
    cur = conn.cursor()  # ❌ no server-side name
    cur.execute(query)

    print("execute")
    if cur.description is None:
        print("⚠️ Query ran, but returned no result set.")
        cur.close()
        conn.close()
        return pl.DataFrame()
    print("description")
    columns = [desc[0] for desc in cur.description]
    rows = cur.fetchall()
    print("fetchall")

    cur.close()
    conn.close()

    if rows:
        # Process rows in batches of 50000
        batch_size = 10000
        dfs = []
        print("loop")
        for i in range(0, len(rows), batch_size):
            batch = rows[i:i + batch_size]
            df_batch = pl.DataFrame(batch, schema=columns, orient="row")
            dfs.append(df_batch)
            print(f"✅ Loaded batch {i//batch_size + 1} with {len(batch):,} rows")
            
        if dfs:
            df = pl.concat(dfs)
            print(f"✅ Total rows loaded into Polars: {df.shape[0]:,}")
            return df
        else:
            print("⚠️ Query returned zero rows.")
            return pl.DataFrame()

In [71]:

max_constituents = 1000
min_volume_eur = 100000
user_country_list = ['US']
user_sector_list = []
user_industry_list = []
user_sector_list = ['Technology']
user_industry_list = ['Software - Application']
selected_stocks = ['']
selected_stocks = "(" + ", ".join(f"'{c}'" for c in selected_stocks) + ")"


perc_list_all = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99, 100]
kpis = {
    'price_to_earnings_ratio_perc': [60, 70, 80, 90, 99, 100],
    'gross_profit_margin_perc': [],
    'net_profit_margin_perc': [],
}

kpi_filters = [
    f"AND {kpi} IN ({', '.join(map(str, values))})"
    for kpi, values in kpis.items() if values
]

kpi_sql = "\n".join(kpi_filters)
active_kpis = [kpi for kpi, values in kpis.items() if values]
kpi_cols        = ", ".join(active_kpis)
prep3_kpi_cols  = ", ".join(f"p3.{kpi}" for kpi in active_kpis)
#prep6_kpi_cols  = ", ".join(f"prep6.{kpi}" for kpi in active_kpis)
prep6_kpi_cols = ", ".join(f"CAST(prep6.{kpi} AS FLOAT8) AS {kpi}" for kpi in active_kpis)



#######

country_query = "SELECT DISTINCT country FROM raw.stock_info WHERE COUNTRY IS NOT NULL"
country_df = run_query(country_query)
country_list_all = list(country_df.country)
country_list_filtered = [country for country in country_list_all if country in user_country_list]
selected_countries = country_list_filtered if country_list_filtered else [''] # country_list_all
countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
#######

sector_query = "SELECT DISTINCT sector FROM raw.stock_info WHERE SECTOR IS NOT NULL"
sector_df = run_query(sector_query)
sector_list_all = list(sector_df.sector)
sector_list_filtered = [sector for sector in sector_list_all if sector in user_sector_list]
selected_sectors = sector_list_filtered if sector_list_filtered else [''] # sector_list_all
sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
#######

industry_query = "SELECT DISTINCT industry FROM raw.stock_info WHERE INDUSTRY IS NOT NULL"
industry_df = run_query(industry_query)
industry_list_all = list(industry_df.industry)
industry_list_filtered = [industry for industry in industry_list_all if industry in user_industry_list]
selected_industries = industry_list_filtered if industry_list_filtered else [''] # industry_list_all
industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
#######



query = f"""
WITH prep1 AS (
    SELECT symbol
    FROM raw.stock_info 
    WHERE (country IN {countries}
      AND industry IN {industries}
      AND sector IN {sectors})
    OR symbol IN {selected_stocks}
),
prep2 AS (
    SELECT *
    FROM clean.financial_metrics_perc
    WHERE 1=1
    {kpi_sql}
),
prep3 AS (
    SELECT 
        p2.symbol, p2.date, p2.fiscal_year, p2.period, p2.reported_currency,
        {kpi_cols}
    FROM prep2 p2
    INNER JOIN prep1 p1 ON p2.symbol = p1.symbol
),
prep4 AS (
    SELECT 
        hmc.*,
        {prep3_kpi_cols}
    FROM raw.historical_market_cap hmc
    INNER JOIN prep3 p3
      ON hmc.symbol = p3.symbol
     AND hmc.year = p3.fiscal_year
     AND hmc.quarter = p3.period
    WHERE hmc.last_quarter_date = TRUE
    OR (hmc.year = EXTRACT(YEAR FROM CURRENT_DATE)
        AND hmc.quarter = 'Q' || EXTRACT(QUARTER FROM CURRENT_DATE)::INT
    )
),
prep5 AS (
    SELECT 
        p4.*, 
        RANK() OVER (
            PARTITION BY p4.year, p4.quarter 
            ORDER BY p4.market_cap_eur DESC
        ) AS mcap_rank
    FROM prep4 p4
),
prep6 AS (
    SELECT *
    FROM prep5
    WHERE mcap_rank <= {max_constituents}
    OR symbol IN {selected_stocks}
),
prep7 AS (
    SELECT *
    FROM raw.historical_price_volume
    WHERE volume_eur > {min_volume_eur}
    OR symbol IN {selected_stocks}
),
prep8 AS (
    SELECT 
        prep7.date,
        prep7.symbol,
        prep7.currency,
        prep7.year,
        prep7.quarter,
        prep7.last_quarter_date,
        CAST(prep7.close as FLOAT8) as close ,
        CAST(prep7.close_eur as FLOAT8) as close_eur,
        CAST(prep7.close_usd as FLOAT8) as close_usd,
        CAST(prep6.market_cap as FLOAT8) as market_cap, 
        CAST(prep6.market_cap_eur as FLOAT8) as market_cap_eur,
        {prep6_kpi_cols},
        CAST(prep6.mcap_rank as INTEGER) as mcap_rank
    FROM prep7
    INNER JOIN prep6
      ON prep7.symbol = prep6.symbol
     AND prep7.year = prep6.year
     AND prep7.quarter = prep6.quarter
)
SELECT *
FROM prep8
--WHERE (EXTRACT(DOW FROM date) = 1 OR last_quarter_date = TRUE)
"""


df = run_query_to_polars_simple(query)
#df = run_query_polars_batched(query)


# constituents_per_day = (
#     df.loc[df["last_quarter_date"] == True]
#       .groupby(["year", "quarter"])["symbol"]
#       .nunique()
#       .reset_index(name="unique_symbol_count")
#       .sort_values(["year", "quarter"], ascending=[False, False])  # both descending
# )


# print(constituents_per_day)


#print(f"Read {len(df)} records from financial_metrics table")
#df[["date", "symbol", "currency", "close", "volume", "year", "quarter", "last_quarter_date", "close_eur", "market_cap_eur"]].head(10)
df.tail()
#print(constituents_per_day)


✅ Query + fetch duration: 0.55 seconds
✅ Query + fetch duration: 0.6 seconds
✅ Query + fetch duration: 0.57 seconds
start
execute
description
fetchall
loop
✅ Loaded batch 1 with 10,000 rows
✅ Loaded batch 2 with 10,000 rows
✅ Loaded batch 3 with 10,000 rows
✅ Loaded batch 4 with 10,000 rows
✅ Loaded batch 5 with 10,000 rows
✅ Loaded batch 6 with 10,000 rows
✅ Loaded batch 7 with 10,000 rows
✅ Loaded batch 8 with 10,000 rows
✅ Loaded batch 9 with 10,000 rows
✅ Loaded batch 10 with 10,000 rows
✅ Loaded batch 11 with 10,000 rows
✅ Loaded batch 12 with 10,000 rows
✅ Loaded batch 13 with 10,000 rows
✅ Loaded batch 14 with 6,035 rows
✅ Total rows loaded into Polars: 136,035


date,symbol,currency,year,quarter,last_quarter_date,close,close_eur,close_usd,market_cap,market_cap_eur,price_to_earnings_ratio_perc,mcap_rank
date,str,str,i64,str,bool,f64,f64,f64,f64,f64,f64,i64
2021-05-25,"""ALU.AX""","""AUD""",2021,"""Q2""",false,27.01,17.1021,20.9485,4.8760e9,3.0849e9,99.0,32
2021-05-26,"""ALU.AX""","""AUD""",2021,"""Q2""",false,27.61,17.5394,21.3835,4.8760e9,3.0849e9,99.0,32
2021-05-27,"""ALU.AX""","""AUD""",2021,"""Q2""",false,28.01,17.7992,21.6896,4.8760e9,3.0849e9,99.0,32
2021-05-28,"""ALU.AX""","""AUD""",2021,"""Q2""",false,28.2,17.8549,21.7559,4.8760e9,3.0849e9,99.0,32
2021-05-31,"""ALU.AX""","""AUD""",2021,"""Q2""",false,28.27,17.8943,21.874,4.8760e9,3.0849e9,99.0,32


In [ ]:

import polars as pl
import os

from datetime import datetime, date
from typing import Union
from utils.utils import run_query, run_query_to_polars_simple


max_constituents = 100
min_volume_eur = 100000
selected_countries = ['US', 'CN']
selected_sectors = ['Technology']
selected_industries = ['Software - Application','Media & Entertainment','Semiconductors', 'Information Technology Services']
selected_stocks = ['']

industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
selected_stocks = "(" + ", ".join(f"'{c}'" for c in selected_stocks) + ")"


perc_list_all = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99, 100]
kpis = {
    'price_to_earnings_ratio_perc': [60, 70, 80, 90, 99, 100],
    'gross_profit_margin_perc': [],
    'net_profit_margin_perc': [],
}

kpi_filters = [
    f"AND {kpi} IN ({', '.join(map(str, values))})"
    for kpi, values in kpis.items() if values
]

kpi_sql = "\n".join(kpi_filters)
active_kpis = [kpi for kpi, values in kpis.items() if values]
kpi_cols        = ", ".join(active_kpis)
prep3_kpi_cols  = ", ".join(f"p3.{kpi}" for kpi in active_kpis)
#prep6_kpi_cols  = ", ".join(f"prep6.{kpi}" for kpi in active_kpis)
prep6_kpi_cols = ", ".join(f"CAST(prep6.{kpi} AS FLOAT8) AS {kpi}" for kpi in active_kpis)



# #######

# country_query = "SELECT DISTINCT country FROM raw.stock_info WHERE COUNTRY IS NOT NULL"
# country_df = run_query(country_query)
# country_list_all = list(country_df.country)
# country_list_filtered = [country for country in country_list_all if country in user_country_list]
# selected_countries = country_list_filtered if country_list_filtered else [''] # country_list_all
# countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
# #######

# sector_query = "SELECT DISTINCT sector FROM raw.stock_info WHERE SECTOR IS NOT NULL"
# sector_df = run_query(sector_query)
# sector_list_all = list(sector_df.sector)
# sector_list_filtered = [sector for sector in sector_list_all if sector in user_sector_list]
# selected_sectors = sector_list_filtered if sector_list_filtered else [''] # sector_list_all
# sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
# #######

# industry_query = "SELECT DISTINCT industry FROM raw.stock_info WHERE INDUSTRY IS NOT NULL"
# industry_df = run_query(industry_query)
# industry_list_all = list(industry_df.industry)
# industry_list_filtered = [industry for industry in industry_list_all if industry in user_industry_list]
# selected_industries = industry_list_filtered if industry_list_filtered else [''] # industry_list_all
# industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
# #######



query = f"""
WITH prep1 AS (
    SELECT symbol
    FROM raw.stock_info 
    WHERE (country IN {countries}
      AND industry IN {industries}
      AND sector IN {sectors})
    OR symbol IN {selected_stocks}
),
prep2 AS (
    SELECT *
    FROM clean.financial_metrics_perc
    WHERE 1=1
    {kpi_sql}
),
prep3 AS (
    SELECT 
        p2.symbol, p2.date, p2.fiscal_year, p2.period, p2.reported_currency,
        {kpi_cols}
    FROM prep2 p2
    INNER JOIN prep1 p1 ON p2.symbol = p1.symbol
),
prep4 AS (
    SELECT 
        hmc.*,
        {prep3_kpi_cols}
    FROM raw.historical_market_cap hmc
    INNER JOIN prep3 p3
      ON hmc.symbol = p3.symbol
     AND hmc.year = p3.fiscal_year
     AND hmc.quarter = p3.period
    WHERE hmc.last_quarter_date = TRUE
    OR (hmc.year = EXTRACT(YEAR FROM CURRENT_DATE)
        AND hmc.quarter = 'Q' || EXTRACT(QUARTER FROM CURRENT_DATE)::INT
    )
),
prep5 AS (
    SELECT 
        p4.*, 
        RANK() OVER (
            PARTITION BY p4.year, p4.quarter 
            ORDER BY p4.market_cap_eur DESC
        ) AS mcap_rank
    FROM prep4 p4
),
prep6 AS (
    SELECT *
    FROM prep5
    WHERE mcap_rank <= {max_constituents}
    OR symbol IN {selected_stocks}
),
prep7 AS (
    SELECT *
    FROM raw.historical_price_volume
    WHERE volume_eur > {min_volume_eur}
    OR symbol IN {selected_stocks}
),
prep8 AS (
    SELECT 
        prep7.date,
        prep7.symbol,
        prep7.currency,
        prep7.year,
        prep7.quarter,
        prep7.last_quarter_date,
        CAST(prep7.close as FLOAT8) as close ,
        CAST(prep7.close_eur as FLOAT8) as close_eur,
        CAST(prep7.close_usd as FLOAT8) as close_usd,
        CAST(prep6.market_cap as FLOAT8) as market_cap, 
        CAST(prep6.market_cap_eur as FLOAT8) as market_cap_eur,
        {prep6_kpi_cols},
        CAST(prep6.mcap_rank as INTEGER) as mcap_rank
    FROM prep7
    INNER JOIN prep6
      ON prep7.symbol = prep6.symbol
     AND prep7.year = prep6.year
     AND prep7.quarter = prep6.quarter
)
SELECT *
FROM prep8
WHERE (EXTRACT(DOW FROM date) = 1 OR last_quarter_date = TRUE)
"""


df = run_query_to_polars_simple(query)
#df = run_query_polars_batched(query)


# constituents_per_day = (
#     df.loc[df["last_quarter_date"] == True]
#       .groupby(["year", "quarter"])["symbol"]
#       .nunique()
#       .reset_index(name="unique_symbol_count")
#       .sort_values(["year", "quarter"], ascending=[False, False])  # both descending
# )


# print(constituents_per_day)


#print(f"Read {len(df)} records from financial_metrics table")
#df[["date", "symbol", "currency", "close", "volume", "year", "quarter", "last_quarter_date", "close_eur", "market_cap_eur"]].head(10)
df.tail()
#print(constituents_per_day)


start
execute
description
fetchall
loop
✅ Loaded batch 1 with 10,000 rows
✅ Loaded batch 2 with 10,000 rows
✅ Loaded batch 3 with 10,000 rows
✅ Loaded batch 4 with 10,000 rows
✅ Loaded batch 5 with 10,000 rows
✅ Loaded batch 6 with 5,929 rows
✅ Total rows loaded into Polars: 55,929


date,symbol,currency,year,quarter,last_quarter_date,close,close_eur,close_usd,market_cap,market_cap_eur,price_to_earnings_ratio_perc,mcap_rank
date,str,str,i64,str,bool,f64,f64,f64,f64,f64,f64,i64
2025-07-14,"""INTU""","""USD""",2025,"""Q3""",false,752.75,645.2733,752.75,2.1074e11,1.8127e11,70.0,100
2025-07-07,"""INTU""","""USD""",2025,"""Q3""",false,783.61,669.2945,783.61,2.1074e11,1.8127e11,70.0,100
2025-07-21,"""INTU""","""USD""",2025,"""Q3""",false,763.69,652.9721,763.69,2.1074e11,1.8127e11,70.0,100
2025-07-28,"""INTU""","""USD""",2025,"""Q3""",false,798.73,689.2318,798.73,2.1074e11,1.8127e11,70.0,100
2025-08-25,"""INTU""","""USD""",2025,"""Q3""",false,656.96,565.3506,656.96,2.1074e11,1.8127e11,70.0,100


In [ ]:

import polars as pl
import os

from datetime import datetime, date
from typing import Union
from utils.utils import run_query, run_query_to_polars_simple


max_constituents = 100
min_volume_eur = 100000
selected_countries = ['US', 'CN']
selected_sectors = ['Technology']
selected_industries = ['Software - Application','Media & Entertainment','Semiconductors', 'Information Technology Services']
selected_stocks = ['']

industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
selected_stocks = "(" + ", ".join(f"'{c}'" for c in selected_stocks) + ")"


perc_list_all = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99, 100]
kpis = {
    'price_to_earnings_ratio_perc': [60, 70, 80, 90, 99, 100],
    'gross_profit_margin_perc': [],
    'net_profit_margin_perc': [],
}

kpi_filters = [
    f"AND {kpi} IN ({', '.join(map(str, values))})"
    for kpi, values in kpis.items() if values
]

kpi_sql = "\n".join(kpi_filters)
active_kpis = [kpi for kpi, values in kpis.items() if values]
kpi_cols        = ", ".join(active_kpis)
prep3_kpi_cols  = ", ".join(f"p3.{kpi}" for kpi in active_kpis)
#prep6_kpi_cols  = ", ".join(f"prep6.{kpi}" for kpi in active_kpis)
prep6_kpi_cols = ", ".join(f"CAST(prep6.{kpi} AS FLOAT8) AS {kpi}" for kpi in active_kpis)



# #######

# country_query = "SELECT DISTINCT country FROM raw.stock_info WHERE COUNTRY IS NOT NULL"
# country_df = run_query(country_query)
# country_list_all = list(country_df.country)
# country_list_filtered = [country for country in country_list_all if country in user_country_list]
# selected_countries = country_list_filtered if country_list_filtered else [''] # country_list_all
# countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
# #######

# sector_query = "SELECT DISTINCT sector FROM raw.stock_info WHERE SECTOR IS NOT NULL"
# sector_df = run_query(sector_query)
# sector_list_all = list(sector_df.sector)
# sector_list_filtered = [sector for sector in sector_list_all if sector in user_sector_list]
# selected_sectors = sector_list_filtered if sector_list_filtered else [''] # sector_list_all
# sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
# #######

# industry_query = "SELECT DISTINCT industry FROM raw.stock_info WHERE INDUSTRY IS NOT NULL"
# industry_df = run_query(industry_query)
# industry_list_all = list(industry_df.industry)
# industry_list_filtered = [industry for industry in industry_list_all if industry in user_industry_list]
# selected_industries = industry_list_filtered if industry_list_filtered else [''] # industry_list_all
# industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
# #######



query = f"""
WITH prep1 AS (
    SELECT symbol
    FROM raw.stock_info 
    WHERE (country IN {countries}
      AND industry IN {industries}
      AND sector IN {sectors})
    OR symbol IN {selected_stocks}
),
prep2 AS (
    SELECT *
    FROM clean.financial_metrics_perc
    WHERE 1=1
    {kpi_sql}
),
prep3 AS (
    SELECT 
        p2.symbol, p2.date, p2.fiscal_year, p2.period, p2.reported_currency,
        {kpi_cols}
    FROM prep2 p2
    INNER JOIN prep1 p1 ON p2.symbol = p1.symbol
),
prep4 AS (
    SELECT 
        hmc.*,
        {prep3_kpi_cols}
    FROM raw.historical_market_cap hmc
    INNER JOIN prep3 p3
      ON hmc.symbol = p3.symbol
     AND hmc.year = p3.fiscal_year
     AND hmc.quarter = p3.period
    WHERE hmc.last_quarter_date = TRUE
    OR (hmc.year = EXTRACT(YEAR FROM CURRENT_DATE)
        AND hmc.quarter = 'Q' || EXTRACT(QUARTER FROM CURRENT_DATE)::INT
    )
),
prep5 AS (
    SELECT 
        p4.*, 
        RANK() OVER (
            PARTITION BY p4.year, p4.quarter 
            ORDER BY p4.market_cap_eur DESC
        ) AS mcap_rank
    FROM prep4 p4
),
prep6 AS (
    SELECT *
    FROM prep5
    WHERE mcap_rank <= {max_constituents}
    OR symbol IN {selected_stocks}
),
prep7 AS (
    SELECT *
    FROM raw.historical_price_volume
    WHERE volume_eur > {min_volume_eur}
    OR symbol IN {selected_stocks}
),
prep8 AS (
    SELECT 
        prep7.date,
        prep7.symbol,
        prep7.currency,
        prep7.year,
        prep7.quarter,
        prep7.last_quarter_date,
        CAST(prep7.close as FLOAT8) as close ,
        CAST(prep7.close_eur as FLOAT8) as close_eur,
        CAST(prep7.close_usd as FLOAT8) as close_usd,
        CAST(prep6.market_cap as FLOAT8) as market_cap, 
        CAST(prep6.market_cap_eur as FLOAT8) as market_cap_eur,
        {prep6_kpi_cols},
        CAST(prep6.mcap_rank as INTEGER) as mcap_rank
    FROM prep7
    INNER JOIN prep6
      ON prep7.symbol = prep6.symbol
     AND prep7.year = prep6.year
     AND prep7.quarter = prep6.quarter
)
SELECT *
FROM prep8
WHERE (EXTRACT(DOW FROM date) = 1 OR last_quarter_date = TRUE)
"""


df = run_query_to_polars_simple(query)
#df = run_query_polars_batched(query)


# constituents_per_day = (
#     df.loc[df["last_quarter_date"] == True]
#       .groupby(["year", "quarter"])["symbol"]
#       .nunique()
#       .reset_index(name="unique_symbol_count")
#       .sort_values(["year", "quarter"], ascending=[False, False])  # both descending
# )


# print(constituents_per_day)


#print(f"Read {len(df)} records from financial_metrics table")
#df[["date", "symbol", "currency", "close", "volume", "year", "quarter", "last_quarter_date", "close_eur", "market_cap_eur"]].head(10)
df.tail()
#print(constituents_per_day)


start
execute
description
fetchall
loop
✅ Loaded batch 1 with 10,000 rows
✅ Loaded batch 2 with 10,000 rows
✅ Loaded batch 3 with 10,000 rows
✅ Loaded batch 4 with 10,000 rows
✅ Loaded batch 5 with 10,000 rows
✅ Loaded batch 6 with 5,929 rows
✅ Total rows loaded into Polars: 55,929


date,symbol,currency,year,quarter,last_quarter_date,close,close_eur,close_usd,market_cap,market_cap_eur,price_to_earnings_ratio_perc,mcap_rank
date,str,str,i64,str,bool,f64,f64,f64,f64,f64,f64,i64
2025-07-14,"""INTU""","""USD""",2025,"""Q3""",false,752.75,645.2733,752.75,2.1074e11,1.8127e11,70.0,100
2025-07-07,"""INTU""","""USD""",2025,"""Q3""",false,783.61,669.2945,783.61,2.1074e11,1.8127e11,70.0,100
2025-07-21,"""INTU""","""USD""",2025,"""Q3""",false,763.69,652.9721,763.69,2.1074e11,1.8127e11,70.0,100
2025-07-28,"""INTU""","""USD""",2025,"""Q3""",false,798.73,689.2318,798.73,2.1074e11,1.8127e11,70.0,100
2025-08-25,"""INTU""","""USD""",2025,"""Q3""",false,656.96,565.3506,656.96,2.1074e11,1.8127e11,70.0,100


In [ ]:

import polars as pl
import os

from datetime import datetime, date
from typing import Union
from utils.utils import run_query, run_query_to_polars_simple
print(os.getenv("POSTGRES_HOST"))p


max_constituents = 100
min_volume_eur = 100000
user_country_list = ['US']
user_sector_list = []
user_industry_list = []
user_sector_list = ['Technology']
user_industry_list = ['Software - Application']
selected_stocks = ['']
selected_stocks = "(" + ", ".join(f"'{c}'" for c in selected_stocks) + ")"


perc_list_all = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99, 100]
kpis = {
    'price_to_earnings_ratio_perc': [60, 70, 80, 90, 99, 100],
    'gross_profit_margin_perc': [],
    'net_profit_margin_perc': [],
}

kpi_filters = [
    f"AND {kpi} IN ({', '.join(map(str, values))})"
    for kpi, values in kpis.items() if values
]

kpi_sql = "\n".join(kpi_filters)
active_kpis = [kpi for kpi, values in kpis.items() if values]
kpi_cols        = ", ".join(active_kpis)
prep3_kpi_cols  = ", ".join(f"p3.{kpi}" for kpi in active_kpis)
#prep6_kpi_cols  = ", ".join(f"prep6.{kpi}" for kpi in active_kpis)
prep6_kpi_cols = ", ".join(f"CAST(prep6.{kpi} AS FLOAT8) AS {kpi}" for kpi in active_kpis)



#######

country_query = "SELECT DISTINCT country FROM raw.stock_info WHERE COUNTRY IS NOT NULL"
country_df = run_query(country_query)
country_list_all = list(country_df.country)
country_list_filtered = [country for country in country_list_all if country in user_country_list]
selected_countries = country_list_filtered if country_list_filtered else [''] # country_list_all
countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
#######

sector_query = "SELECT DISTINCT sector FROM raw.stock_info WHERE SECTOR IS NOT NULL"
sector_df = run_query(sector_query)
sector_list_all = list(sector_df.sector)
sector_list_filtered = [sector for sector in sector_list_all if sector in user_sector_list]
selected_sectors = sector_list_filtered if sector_list_filtered else [''] # sector_list_all
sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
#######

industry_query = "SELECT DISTINCT industry FROM raw.stock_info WHERE INDUSTRY IS NOT NULL"
industry_df = run_query(industry_query)
industry_list_all = list(industry_df.industry)
industry_list_filtered = [industry for industry in industry_list_all if industry in user_industry_list]
selected_industries = industry_list_filtered if industry_list_filtered else [''] # industry_list_all
industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
#######



query = f"""
WITH prep1 AS (
    SELECT symbol
    FROM raw.stock_info 
    WHERE (country IN {countries}
      AND industry IN {industries}
      AND sector IN {sectors})
    OR symbol IN {selected_stocks}
),
prep2 AS (
    SELECT *
    FROM clean.financial_metrics_perc
    WHERE 1=1
    {kpi_sql}
),
prep3 AS (
    SELECT 
        p2.symbol, p2.date, p2.fiscal_year, p2.period, p2.reported_currency,
        {kpi_cols}
    FROM prep2 p2
    INNER JOIN prep1 p1 ON p2.symbol = p1.symbol
),
prep4 AS (
    SELECT 
        hmc.*,
        {prep3_kpi_cols}
    FROM raw.historical_market_cap hmc
    INNER JOIN prep3 p3
      ON hmc.symbol = p3.symbol
     AND hmc.year = p3.fiscal_year
     AND hmc.quarter = p3.period
    WHERE hmc.last_quarter_date = TRUE
    OR (hmc.year = EXTRACT(YEAR FROM CURRENT_DATE)
        AND hmc.quarter = 'Q' || EXTRACT(QUARTER FROM CURRENT_DATE)::INT
    )
),
prep5 AS (
    SELECT 
        p4.*, 
        RANK() OVER (
            PARTITION BY p4.year, p4.quarter 
            ORDER BY p4.market_cap_eur DESC
        ) AS mcap_rank
    FROM prep4 p4
),
prep6 AS (
    SELECT *
    FROM prep5
    WHERE mcap_rank <= {max_constituents}
    OR symbol IN {selected_stocks}
),
prep7 AS (
    SELECT *
    FROM raw.historical_price_volume
    WHERE volume_eur > {min_volume_eur}
    OR symbol IN {selected_stocks}
),
prep8 AS (
    SELECT 
        prep7.date,
        prep7.symbol,
        prep7.currency,
        prep7.year,
        prep7.quarter,
        prep7.last_quarter_date,
        CAST(prep7.close as FLOAT8) as close ,
        CAST(prep7.close_eur as FLOAT8) as close_eur,
        CAST(prep7.close_usd as FLOAT8) as close_usd,
        CAST(prep6.market_cap as FLOAT8) as market_cap, 
        CAST(prep6.market_cap_eur as FLOAT8) as market_cap_eur,
        {prep6_kpi_cols},
        CAST(prep6.mcap_rank as INTEGER) as mcap_rank
    FROM prep7
    INNER JOIN prep6
      ON prep7.symbol = prep6.symbol
     AND prep7.year = prep6.year
     AND prep7.quarter = prep6.quarter
)
SELECT *
FROM prep8
WHERE (EXTRACT(DOW FROM date) = 1 OR last_quarter_date = TRUE)
"""


df = run_query_to_polars_simple(query)
#df = run_query_polars_batched(query)


# constituents_per_day = (
#     df.loc[df["last_quarter_date"] == True]
#       .groupby(["year", "quarter"])["symbol"]
#       .nunique()
#       .reset_index(name="unique_symbol_count")
#       .sort_values(["year", "quarter"], ascending=[False, False])  # both descending
# )


# print(constituents_per_day)


#print(f"Read {len(df)} records from financial_metrics table")
#df[["date", "symbol", "currency", "close", "volume", "year", "quarter", "last_quarter_date", "close_eur", "market_cap_eur"]].head(10)
df.tail()
#print(constituents_per_day)


91.107.196.130
✅ Query + fetch duration: 0.56 seconds
✅ Query + fetch duration: 0.63 seconds
✅ Query + fetch duration: 0.61 seconds
start
execute
description
fetchall
loop
✅ Loaded batch 1 with 10,000 rows
✅ Loaded batch 2 with 10,000 rows
✅ Loaded batch 3 with 2,346 rows
✅ Total rows loaded into Polars: 22,346


date,symbol,currency,year,quarter,last_quarter_date,close,close_eur,close_usd,market_cap,market_cap_eur,price_to_earnings_ratio_perc,mcap_rank
date,str,str,i64,str,bool,f64,f64,f64,f64,f64,f64,i64
2025-07-14,"""WDAY""","""USD""",2025,"""Q3""",false,224.76,192.669,224.76,6.3441e10,5.4744e10,99.0,100
2025-07-07,"""WDAY""","""USD""",2025,"""Q3""",false,241.21,206.0215,241.21,6.3441e10,5.4744e10,99.0,100
2025-07-21,"""WDAY""","""USD""",2025,"""Q3""",false,233.71,199.8273,233.71,6.3441e10,5.4744e10,99.0,100
2025-07-28,"""WDAY""","""USD""",2025,"""Q3""",false,238.04,205.407,238.04,6.3441e10,5.4744e10,99.0,100
2025-08-25,"""WDAY""","""USD""",2025,"""Q3""",false,223.39,192.2395,223.39,6.3441e10,5.4744e10,99.0,100


In [ ]:

import polars as pl
import os

from datetime import datetime, date
from typing import Union
from utils.utils import run_query, run_query_to_polars_simple
print(os.getenv("POSTGRES_HOST"))p


max_constituents = 100
min_volume_eur = 100000
user_country_list = ['US']
user_sector_list = []
user_industry_list = []
user_sector_list = ['Technology']
user_industry_list = ['Software - Application']
selected_stocks = ['']
selected_stocks = "(" + ", ".join(f"'{c}'" for c in selected_stocks) + ")"


perc_list_all = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99, 100]
kpis = {
    'price_to_earnings_ratio_perc': [60, 70, 80, 90, 99, 100],
    'gross_profit_margin_perc': [],
    'net_profit_margin_perc': [],
}

kpi_filters = [
    f"AND {kpi} IN ({', '.join(map(str, values))})"
    for kpi, values in kpis.items() if values
]

kpi_sql = "\n".join(kpi_filters)
active_kpis = [kpi for kpi, values in kpis.items() if values]
kpi_cols        = ", ".join(active_kpis)
prep3_kpi_cols  = ", ".join(f"p3.{kpi}" for kpi in active_kpis)
#prep6_kpi_cols  = ", ".join(f"prep6.{kpi}" for kpi in active_kpis)
prep6_kpi_cols = ", ".join(f"CAST(prep6.{kpi} AS FLOAT8) AS {kpi}" for kpi in active_kpis)



#######

country_query = "SELECT DISTINCT country FROM raw.stock_info WHERE COUNTRY IS NOT NULL"
country_df = run_query(country_query)
country_list_all = list(country_df.country)
country_list_filtered = [country for country in country_list_all if country in user_country_list]
selected_countries = country_list_filtered if country_list_filtered else [''] # country_list_all
countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
#######

sector_query = "SELECT DISTINCT sector FROM raw.stock_info WHERE SECTOR IS NOT NULL"
sector_df = run_query(sector_query)
sector_list_all = list(sector_df.sector)
sector_list_filtered = [sector for sector in sector_list_all if sector in user_sector_list]
selected_sectors = sector_list_filtered if sector_list_filtered else [''] # sector_list_all
sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
#######

industry_query = "SELECT DISTINCT industry FROM raw.stock_info WHERE INDUSTRY IS NOT NULL"
industry_df = run_query(industry_query)
industry_list_all = list(industry_df.industry)
industry_list_filtered = [industry for industry in industry_list_all if industry in user_industry_list]
selected_industries = industry_list_filtered if industry_list_filtered else [''] # industry_list_all
industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
#######



query = f"""
WITH prep1 AS (
    SELECT symbol
    FROM raw.stock_info 
    WHERE (country IN {countries}
      AND industry IN {industries}
      AND sector IN {sectors})
    OR symbol IN {selected_stocks}
),
prep2 AS (
    SELECT *
    FROM clean.financial_metrics_perc
    WHERE 1=1
    {kpi_sql}
),
prep3 AS (
    SELECT 
        p2.symbol, p2.date, p2.fiscal_year, p2.period, p2.reported_currency,
        {kpi_cols}
    FROM prep2 p2
    INNER JOIN prep1 p1 ON p2.symbol = p1.symbol
),
prep4 AS (
    SELECT 
        hmc.*,
        {prep3_kpi_cols}
    FROM raw.historical_market_cap hmc
    INNER JOIN prep3 p3
      ON hmc.symbol = p3.symbol
     AND hmc.year = p3.fiscal_year
     AND hmc.quarter = p3.period
    WHERE hmc.last_quarter_date = TRUE
    OR (hmc.year = EXTRACT(YEAR FROM CURRENT_DATE)
        AND hmc.quarter = 'Q' || EXTRACT(QUARTER FROM CURRENT_DATE)::INT
    )
),
prep5 AS (
    SELECT 
        p4.*, 
        RANK() OVER (
            PARTITION BY p4.year, p4.quarter 
            ORDER BY p4.market_cap_eur DESC
        ) AS mcap_rank
    FROM prep4 p4
),
prep6 AS (
    SELECT *
    FROM prep5
    WHERE mcap_rank <= {max_constituents}
    OR symbol IN {selected_stocks}
),
prep7 AS (
    SELECT *
    FROM raw.historical_price_volume
    WHERE volume_eur > {min_volume_eur}
    OR symbol IN {selected_stocks}
),
prep8 AS (
    SELECT 
        prep7.date,
        prep7.symbol,
        prep7.currency,
        prep7.year,
        prep7.quarter,
        prep7.last_quarter_date,
        CAST(prep7.close as FLOAT8) as close ,
        CAST(prep7.close_eur as FLOAT8) as close_eur,
        CAST(prep7.close_usd as FLOAT8) as close_usd,
        CAST(prep6.market_cap as FLOAT8) as market_cap, 
        CAST(prep6.market_cap_eur as FLOAT8) as market_cap_eur,
        {prep6_kpi_cols},
        CAST(prep6.mcap_rank as INTEGER) as mcap_rank
    FROM prep7
    INNER JOIN prep6
      ON prep7.symbol = prep6.symbol
     AND prep7.year = prep6.year
     AND prep7.quarter = prep6.quarter
)
SELECT *
FROM prep8
WHERE (EXTRACT(DOW FROM date) = 1 OR last_quarter_date = TRUE)
"""


df = run_query_to_polars_simple(query)
#df = run_query_polars_batched(query)


# constituents_per_day = (
#     df.loc[df["last_quarter_date"] == True]
#       .groupby(["year", "quarter"])["symbol"]
#       .nunique()
#       .reset_index(name="unique_symbol_count")
#       .sort_values(["year", "quarter"], ascending=[False, False])  # both descending
# )


# print(constituents_per_day)


#print(f"Read {len(df)} records from financial_metrics table")
#df[["date", "symbol", "currency", "close", "volume", "year", "quarter", "last_quarter_date", "close_eur", "market_cap_eur"]].head(10)
df.tail()
#print(constituents_per_day)


91.107.196.130
✅ Query + fetch duration: 0.56 seconds
✅ Query + fetch duration: 0.63 seconds
✅ Query + fetch duration: 0.61 seconds
start
execute
description
fetchall
loop
✅ Loaded batch 1 with 10,000 rows
✅ Loaded batch 2 with 10,000 rows
✅ Loaded batch 3 with 2,346 rows
✅ Total rows loaded into Polars: 22,346


date,symbol,currency,year,quarter,last_quarter_date,close,close_eur,close_usd,market_cap,market_cap_eur,price_to_earnings_ratio_perc,mcap_rank
date,str,str,i64,str,bool,f64,f64,f64,f64,f64,f64,i64
2025-07-14,"""WDAY""","""USD""",2025,"""Q3""",false,224.76,192.669,224.76,6.3441e10,5.4744e10,99.0,100
2025-07-07,"""WDAY""","""USD""",2025,"""Q3""",false,241.21,206.0215,241.21,6.3441e10,5.4744e10,99.0,100
2025-07-21,"""WDAY""","""USD""",2025,"""Q3""",false,233.71,199.8273,233.71,6.3441e10,5.4744e10,99.0,100
2025-07-28,"""WDAY""","""USD""",2025,"""Q3""",false,238.04,205.407,238.04,6.3441e10,5.4744e10,99.0,100
2025-08-25,"""WDAY""","""USD""",2025,"""Q3""",false,223.39,192.2395,223.39,6.3441e10,5.4744e10,99.0,100


In [ ]:

import polars as pl
import os

from datetime import datetime, date
from typing import Union
from utils.utils import run_query, run_query_to_polars_simple
print(os.getenv("POSTGRES_HOST"))p


max_constituents = 100
min_volume_eur = 100000
user_country_list = ['US']
user_sector_list = []
user_industry_list = []
user_sector_list = ['Technology']
user_industry_list = ['Software - Application']
selected_stocks = ['']
selected_stocks = "(" + ", ".join(f"'{c}'" for c in selected_stocks) + ")"


perc_list_all = [1, 10, 20, 30, 40, 50, 60, 70, 80, 90, 99, 100]
kpis = {
    'price_to_earnings_ratio_perc': [60, 70, 80, 90, 99, 100],
    'gross_profit_margin_perc': [],
    'net_profit_margin_perc': [],
}

kpi_filters = [
    f"AND {kpi} IN ({', '.join(map(str, values))})"
    for kpi, values in kpis.items() if values
]

kpi_sql = "\n".join(kpi_filters)
active_kpis = [kpi for kpi, values in kpis.items() if values]
kpi_cols        = ", ".join(active_kpis)
prep3_kpi_cols  = ", ".join(f"p3.{kpi}" for kpi in active_kpis)
#prep6_kpi_cols  = ", ".join(f"prep6.{kpi}" for kpi in active_kpis)
prep6_kpi_cols = ", ".join(f"CAST(prep6.{kpi} AS FLOAT8) AS {kpi}" for kpi in active_kpis)



#######

country_query = "SELECT DISTINCT country FROM raw.stock_info WHERE COUNTRY IS NOT NULL"
country_df = run_query(country_query)
country_list_all = list(country_df.country)
country_list_filtered = [country for country in country_list_all if country in user_country_list]
selected_countries = country_list_filtered if country_list_filtered else [''] # country_list_all
countries = "(" + ", ".join(f"'{c}'" for c in selected_countries) + ")"
#######

sector_query = "SELECT DISTINCT sector FROM raw.stock_info WHERE SECTOR IS NOT NULL"
sector_df = run_query(sector_query)
sector_list_all = list(sector_df.sector)
sector_list_filtered = [sector for sector in sector_list_all if sector in user_sector_list]
selected_sectors = sector_list_filtered if sector_list_filtered else [''] # sector_list_all
sectors = "(" + ", ".join(f"'{s}'" for s in selected_sectors) + ")"
#######

industry_query = "SELECT DISTINCT industry FROM raw.stock_info WHERE INDUSTRY IS NOT NULL"
industry_df = run_query(industry_query)
industry_list_all = list(industry_df.industry)
industry_list_filtered = [industry for industry in industry_list_all if industry in user_industry_list]
selected_industries = industry_list_filtered if industry_list_filtered else [''] # industry_list_all
industries = "(" + ", ".join(f"'{i}'" for i in selected_industries) + ")"
#######



query = f"""
WITH prep1 AS (
    SELECT symbol
    FROM raw.stock_info 
    WHERE (country IN {countries}
      AND industry IN {industries}
      AND sector IN {sectors})
    OR symbol IN {selected_stocks}
),
prep2 AS (
    SELECT *
    FROM clean.financial_metrics_perc
    WHERE 1=1
    {kpi_sql}
),
prep3 AS (
    SELECT 
        p2.symbol, p2.date, p2.fiscal_year, p2.period, p2.reported_currency,
        {kpi_cols}
    FROM prep2 p2
    INNER JOIN prep1 p1 ON p2.symbol = p1.symbol
),
prep4 AS (
    SELECT 
        hmc.*,
        {prep3_kpi_cols}
    FROM raw.historical_market_cap hmc
    INNER JOIN prep3 p3
      ON hmc.symbol = p3.symbol
     AND hmc.year = p3.fiscal_year
     AND hmc.quarter = p3.period
    WHERE hmc.last_quarter_date = TRUE
    OR (hmc.year = EXTRACT(YEAR FROM CURRENT_DATE)
        AND hmc.quarter = 'Q' || EXTRACT(QUARTER FROM CURRENT_DATE)::INT
    )
),
prep5 AS (
    SELECT 
        p4.*, 
        RANK() OVER (
            PARTITION BY p4.year, p4.quarter 
            ORDER BY p4.market_cap_eur DESC
        ) AS mcap_rank
    FROM prep4 p4
),
prep6 AS (
    SELECT *
    FROM prep5
    WHERE mcap_rank <= {max_constituents}
    OR symbol IN {selected_stocks}
),
prep7 AS (
    SELECT *
    FROM raw.historical_price_volume
    WHERE volume_eur > {min_volume_eur}
    OR symbol IN {selected_stocks}
),
prep8 AS (
    SELECT 
        prep7.date,
        prep7.symbol,
        prep7.currency,
        prep7.year,
        prep7.quarter,
        prep7.last_quarter_date,
        CAST(prep7.close as FLOAT8) as close ,
        CAST(prep7.close_eur as FLOAT8) as close_eur,
        CAST(prep7.close_usd as FLOAT8) as close_usd,
        CAST(prep6.market_cap as FLOAT8) as market_cap, 
        CAST(prep6.market_cap_eur as FLOAT8) as market_cap_eur,
        {prep6_kpi_cols},
        CAST(prep6.mcap_rank as INTEGER) as mcap_rank
    FROM prep7
    INNER JOIN prep6
      ON prep7.symbol = prep6.symbol
     AND prep7.year = prep6.year
     AND prep7.quarter = prep6.quarter
)
SELECT *
FROM prep8
WHERE (EXTRACT(DOW FROM date) = 1 OR last_quarter_date = TRUE)
"""


df = run_query_to_polars_simple(query)
#df = run_query_polars_batched(query)


# constituents_per_day = (
#     df.loc[df["last_quarter_date"] == True]
#       .groupby(["year", "quarter"])["symbol"]
#       .nunique()
#       .reset_index(name="unique_symbol_count")
#       .sort_values(["year", "quarter"], ascending=[False, False])  # both descending
# )


# print(constituents_per_day)


#print(f"Read {len(df)} records from financial_metrics table")
#df[["date", "symbol", "currency", "close", "volume", "year", "quarter", "last_quarter_date", "close_eur", "market_cap_eur"]].head(10)
df.tail()
#print(constituents_per_day)


91.107.196.130
✅ Query + fetch duration: 0.56 seconds
✅ Query + fetch duration: 0.63 seconds
✅ Query + fetch duration: 0.61 seconds
start
execute
description
fetchall
loop
✅ Loaded batch 1 with 10,000 rows
✅ Loaded batch 2 with 10,000 rows
✅ Loaded batch 3 with 2,346 rows
✅ Total rows loaded into Polars: 22,346


date,symbol,currency,year,quarter,last_quarter_date,close,close_eur,close_usd,market_cap,market_cap_eur,price_to_earnings_ratio_perc,mcap_rank
date,str,str,i64,str,bool,f64,f64,f64,f64,f64,f64,i64
2025-07-14,"""WDAY""","""USD""",2025,"""Q3""",false,224.76,192.669,224.76,6.3441e10,5.4744e10,99.0,100
2025-07-07,"""WDAY""","""USD""",2025,"""Q3""",false,241.21,206.0215,241.21,6.3441e10,5.4744e10,99.0,100
2025-07-21,"""WDAY""","""USD""",2025,"""Q3""",false,233.71,199.8273,233.71,6.3441e10,5.4744e10,99.0,100
2025-07-28,"""WDAY""","""USD""",2025,"""Q3""",false,238.04,205.407,238.04,6.3441e10,5.4744e10,99.0,100
2025-08-25,"""WDAY""","""USD""",2025,"""Q3""",false,223.39,192.2395,223.39,6.3441e10,5.4744e10,99.0,100


In [67]:
df.select(pl.col("date").max())

date
date
2025-08-27


In [ ]:


def make_index(df: pl.DataFrame, index_start_date: Union[date, str, None] = "2015-03-15") -> pl.DataFrame:
    # Step 0: Cast decimals to floats
    decimal_cols = [name for name, dtype in zip(df.columns, df.dtypes) if dtype.base_type() == pl.Decimal]
    df = df.with_columns([pl.col(c).cast(pl.Float64) for c in decimal_cols])

    # Step 1: Parse date column
    if df.schema["date"] == pl.Utf8:
        df = df.with_columns(pl.col("date").str.to_date())
    elif df.schema["date"] != pl.Date:
        df = df.with_columns(pl.col("date").cast(pl.Date))

    # Step 2: Forward-fill prices
    df = (
        df.sort(["symbol", "date"])
          .with_columns([
              pl.col("close_eur").forward_fill().over("symbol")
          ])
    )

    # Step 3: Rebalance snapshots
    rebalance_df = df.filter(pl.col("last_quarter_date") == True)

    rebalance_snapshots = (
        rebalance_df
        .group_by(["year", "quarter"])
        .agg([
            pl.first("date").alias("rebalance_date"),
            pl.col("symbol"),
            pl.col("market_cap_eur")
        ])
    )

    # Step 4: Parse index_start_date
    if isinstance(index_start_date, str):
        index_start_date = datetime.strptime(index_start_date, "%Y-%m-%d").date()

    # Step 5: Find the latest rebalance on or before index_start_date
    initial_snapshot = rebalance_snapshots.filter(pl.col("rebalance_date") <= index_start_date)
    if initial_snapshot.is_empty():
        raise ValueError(f"No rebalance snapshot found on or before {index_start_date}")

    initial_row = initial_snapshot.sort("rebalance_date", descending=True).row(0, named=True)
    initial_weights = {
        s: float(m) / float(sum(initial_row["market_cap_eur"]))
        for s, m in zip(initial_row["symbol"], initial_row["market_cap_eur"])
    }
    active_weights = initial_weights
    last_rebalance_date = initial_row["rebalance_date"]

    # Step 6: Store future quarterly rebalance dates > index_start_date
    future_rebalances = {
        row["rebalance_date"]: {
            s: float(m) / float(sum(row["market_cap_eur"]))
            for s, m in zip(row["symbol"], row["market_cap_eur"])
        }
        for row in rebalance_snapshots.iter_rows(named=True)
        if row["rebalance_date"] > index_start_date
    }

    # Step 7: Pivot prices
    pivot = (
        df.with_columns([
            pl.col("date").cast(pl.Utf8)
        ])
        .select(["date", "symbol", "close_eur"])
        .pivot(index="date", values="close_eur", on="symbol", aggregate_function="first")
        .with_columns([
            pl.col("date").str.to_date()
        ])
        .sort("date")
        .with_columns(
            pl.all().exclude("date").fill_null(strategy="forward")
        )
    )

    pivot = pivot.filter(pl.col("date") >= pl.lit(index_start_date))

    # Step 8: Main loop
    index_value = 1000.0
    index_values = []
    previous_prices = None
    symbol_columns = [c for c in pivot.columns if c != "date"]

    for row in pivot.iter_rows(named=True):
        current_date = row["date"]
        price_row = {s: row[s] for s in symbol_columns}

        # Check if rebalancing today
        if current_date in future_rebalances:
            active_weights = future_rebalances[current_date]
            previous_prices = {s: price_row[s] for s in active_weights if price_row[s] is not None}
            index_values.append((current_date, index_value))
            continue

        # Initialize previous prices on the start date
        if previous_prices is None:
            previous_prices = {s: price_row[s] for s in active_weights if price_row[s] is not None}
            index_values.append((current_date, index_value))
            continue

        # Compute daily return
        returns = {}
        for symbol, prev_price in previous_prices.items():
            current_price = price_row.get(symbol)
            if current_price is not None and prev_price > 0:
                returns[symbol] = current_price / prev_price
            else:
                returns[symbol] = 1.0

        daily_return = sum(
            active_weights[s] * returns.get(s, 1.0) for s in active_weights
        )
        index_value *= daily_return

        # Update previous prices
        for s in previous_prices:
            if price_row.get(s) is not None:
                previous_prices[s] = price_row[s]

        index_values.append((current_date, index_value))

    return pl.DataFrame(index_values, schema=["date", "index_value"], orient="row").sort("date")

import polars as pl
pl.Config.set_tbl_rows(-1)   # show all rows
pl.Config.set_tbl_cols(-1) 
index_df = make_index(df, index_start_date="2014-01-13")
index_df.sort("date")

date,index_value
date,f64
2014-01-13,1000.0
2014-01-14,1014.710618
2014-01-15,1032.113359
2014-01-16,1033.357672
2014-01-17,1034.36053
2014-01-21,1035.707152
2014-01-22,1034.56588
2014-01-23,1020.94378
2014-01-24,1000.517641
